In [1]:
import pandas as pd
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict
import csv

In [13]:
players = pd.read_csv('./data/players.csv')
games = pd.read_csv('./data/games.csv')
plays = pd.read_csv('./data/plays.csv')
tackle_scores = pd.read_csv('./intermediate_data/tackle_acc_and_totals.csv')
AVG_TACKLE_SCORE = np.average(tackle_scores.score)
AVG_TACKLE_SCORE

0.11976818934459971

In [4]:
def getClosest(players, football):
    distances = []
    for index, row in players.iterrows():
        distance = np.linalg.norm(np.subtract([football.x.values[0], football.y.values[0]], [row.x, row.y]))
        distances.append((row.nflId, distance))
    closest_id, closest_dist = sorted(distances, key= lambda t: t[1])[0]
    return closest_id, closest_dist

In [15]:
closest_defender = defaultdict(dict)
for week_num in range(1, 18):
    print(week_num)
    g_week = games[games['week'] == week_num]
    week_str = './data/week' + str(week_num) + '.csv'
    current_week_data = pd.read_csv(week_str)
    
    for _, g_row in g_week.iterrows():
        GAMEID = g_row.gameId
        cwg_data = current_week_data[current_week_data['gameId'] == GAMEID]
        pg = plays[plays['gameId'] == GAMEID]
        
        for _, p_row in pg.iterrows():
            
            if p_row.passResult == 'C':
                PLAYID = p_row.playId
                curr_play_data = cwg_data[cwg_data['playId'] == PLAYID]
                
                play_events = pd.unique(curr_play_data.event)                     
                if 'pass_outcome_caught' in play_events:
                    try:
                        OFFENSIVE_TEAM = curr_play_data[curr_play_data['position'] == 'QB'].team.values[0]
                        DEFENSIVE_TEAM = ('home' if OFFENSIVE_TEAM == 'away' else 'away')
                    except:
                        continue
                    
                    pc_data = curr_play_data[curr_play_data['event'] == 'pass_outcome_caught']
                    football1 = pc_data[pc_data['team'] == 'football']
                    offensive_players = pc_data[pc_data['team'] == OFFENSIVE_TEAM]
                    defensive_players = pc_data[pc_data['team'] == DEFENSIVE_TEAM]
                    try:
                        closest1 = getClosest(offensive_players, football1)[0]
                    except:
                        continue
                        
                    receiver = pc_data[pc_data['nflId'] == closest1]
                    try:
                        closest_id, closest_dist = getClosest(defensive_players, receiver)
                    except:
                        continue
#                     print(closest_id, closest_dist)
                    
                    if closest_id in list(tackle_scores.nflId):
                        t_sc = tackle_scores[tackle_scores['nflId'] == closest_id].score.values[0]
                    else:
                        t_sc = AVG_TACKLE_SCORE
                    
                    closest_defender[GAMEID][PLAYID] = (closest_dist, t_sc)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [19]:
with open('./intermediate_data/closest_defender.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['gameId', 'playId', 'closest_distance', 'closest_score'])
    for GAME in closest_defender:
        for PLAY, val in closest_defender[GAME].items():
            writer.writerow([GAME, PLAY, val[0], val[1]])